# Installing and Importing Dependencies

In [4]:
!pip install pycaw

  Using cached pycaw-20230407-py3-none-any.whl (24 kB)
  Obtaining dependency information for comtypes from https://files.pythonhosted.org/packages/c2/a7/fe4bd49b5c4afa7a7ed3852abda6909e48c00715e6a134e47055381113aa/comtypes-1.2.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/184.3 kB ? eta -:--:--
   ------------------------ --------------- 112.6/184.3 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 184.3/184.3 kB 3.7 MB/s eta 0:00:00


In [2]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [ ]:
wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0
detector = htm.handDetector(detectionCon = 0.8)

#pycaw
devices = AudioUtilities.GetSpeakers()   #initializations
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
#volume.SetMasterVolumeLevel(-20.0, None) #%25 % volume
#volume.SetMasterVolumeLevel(-5.0, None) #75% volume
#volume.SetMasterVolumeLevel(0, None) #100% volume
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
#volBar = 240

while True:
    success, img = cap.read()
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.flip(img, 1)  # Flip the image horizontally
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw = False)
    if len(lmList) != 0:
        #print(lmList[4], lmList[8])
        
        #to get x and y coordinates of x and y axes
        x1, y1 = lmList[4][1], lmList[4][2] 
        x2, y2 = lmList[8][1], lmList[8][2]
        
        cx, cy = (x1 + x2)//2, (y1 + y2)//2 #midpoint of the drawn line between the thumb and the index finger
        
        #drawing a circle on the selected points namely here thumb and the index finger
        cv2.circle(img, (x1, y1), 10, (255, 0, 255), cv2.FILLED) 
        cv2.circle(img, (x2, y2), 10, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3) #drawing a line between the thumb and the index finger
        cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED)
        
        length = math.hypot(x2 - x1, y2 - y1)
        #print(length)
        
        #convert hand length to volume length
        vol = np.interp(length, [50, 100], [minVol, maxVol])
        #volBar = np.interp(length, [20, 140], [240, 120])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None) 

        if length < 50:
            cv2.circle(img, (cx, cy), 10, (0, 255, 0), cv2.FILLED)
    #cv2.rectangle(img, (50, 150), (85, 400), (0, 255, 0), 3)
    #cv2.rectangle(img, (50, int(vol)), (85, 400), (0, 255, 0), cv2.FILLED)
    
    # Draw volume bar
    volBar = np.interp(vol, [minVol, maxVol], [400, 150])
    cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (0, 0, 255), cv2.FILLED)
    
    # Calculate volume percentage
    volPercent = np.interp(vol, [minVol, maxVol], [0, 100])

    # Draw volume percentage on the image
    cv2.putText(img, f'Volume: {int(volPercent)}%', (40, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)
    
    #get fps and display it
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (40,50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0 , 0), 3)
    
    cv2.imshow("Image", img)
    key = cv2.waitKey(1) & 0xFF
    if key == 27: #27 is the ASCII code for escape key
        break
cap.release()
cv2.destroyAllWindows()

105 0.0
102 0.0
104 0.0
107 0.0
104 0.0
104 0.0
107 0.0
104 0.0
104 0.0
99 -0.18894120797344982
101 0.0
100 0.0
99 -0.05552082812403114
99 -0.736262721589199
82 -12.97803674085862
83 -12.388611442363999
82 -13.063347646921073
83 -12.388611442363999
53 -34.218547035081784
51 -36.252745795223724
54 -33.6038615706897
24 -37.0
24 -37.0
24 -37.0
18 -37.0
18 -37.0
17 -37.0
19 -37.0
18 -37.0
18 -37.0
18 -37.0
18 -37.0
17 -37.0
16 -37.0
19 -37.0
18 -37.0
16 -37.0
18 -37.0
18 -37.0
17 -37.0
17 -37.0
17 -37.0
17 -37.0
18 -37.0
18 -37.0
16 -37.0
17 -37.0
16 -37.0
17 -37.0
17 -37.0
17 -37.0
65 -25.678509956749053
68 -23.68
67 -24.023969745486987
87 -9.467078169356075
87 -9.128542485928378
83 -12.007606918267783
108 0.0
107 0.0
147 0.0
161 0.0
159 0.0
158 0.0
167 0.0
168 0.0
168 0.0
173 0.0
174 0.0
175 0.0
173 0.0
173 0.0
174 0.0
173 0.0
173 0.0
173 0.0
172 0.0
171 0.0
171 0.0
168 0.0
168 0.0
168 0.0
170 0.0
170 0.0
168 0.0
168 0.0
168 0.0
168 0.0
166 0.0
166 0.0
167 0.0
167 0.0
167 0.0
166 0.0
172